# ESMFold breakdown
It took ouput from ESM (sequence reperesentation, attention matrix) as input
into folding trunk then structure module to predict structure.  

utility modules (misc.py)
read seqence module (encode_sequence): input sequence, add chain_linker( optional, default 'G'*25), add residue_index_offset (add a jump in index if in different chain)
return:encoded (residue token), residx, linker_mask, chain_index

read seqence in batch 

output_to_pdb: input: output from model (in dict format) output: pdb files

Attention: to calcualte attention in Transformer

Dropout: 

SequencToPair:

PairToSequence:





## This is how differnt ESM model get loaded into ESMFold model

In [1]:
def _load_model(model_name):
    if model_name.endswith(".pt"):  # local, treat as filepath
        model_path = Path(model_name)
        model_data = torch.load(str(model_path), map_location="cpu")
    else:  # load from hub
        url = f"https://dl.fbaipublicfiles.com/fair-esm/models/{model_name}.pt"
        model_data = torch.hub.load_state_dict_from_url(url, progress=False, map_location="cpu")

    cfg = model_data["cfg"]["model"]
    model_state = model_data["model"]  # contain weights and bais
    model = ESMFold(esmfold_config=cfg)

    expected_keys = set(model.state_dict().keys())
    found_keys = set(model_state.keys())

    missing_essential_keys = []  # make sure keys are compatiable before load parameters
    for missing_key in expected_keys - found_keys:
        if not missing_key.startswith("esm."):
            missing_essential_keys.append(missing_key)

    if missing_essential_keys:
        raise RuntimeError(f"Keys '{', '.join(missing_essential_keys)}' are missing.")

    model.load_state_dict(model_state, strict=False) # 

    return model

## Triangluar self attention block
Inputs:
sequence_state: Shape (B, L, sequence_state_dim) (batch, sequence length, feature dimension).  
pairwise_state: Shape (B, L, L, pairwise_state_dim) (batch, pairwise interactions, feature dimension).  
mask: Optional (B, L) mask indicating valid sequence positions.  

operation:
Sequence State Updates:
Applies pair-to-sequence projection (self.pair_to_sequence) to derive biases for attention.
Applies self-attention (self.seq_attention) with the computed biases.
Adds residual connections and applies an MLP (self.mlp_seq) to the sequence state.
Pairwise State Updates:

Updates pairwise state with sequence-to-pair projections (self.sequence_to_pair).
Applies triangular multiplications (outgoing and incoming).
Applies triangular attention (starting and ending nodes).
Adds residual connections and applies an MLP (self.mlp_pair) to the pairwise state.
Returns the updated sequence and pairwise states.


In [2]:
# Code fore sequence_to_pair
import torch
import torch.nn as nn

class SequenceToPair(nn.Module):
    def __init__(self, sequence_state_dim, inner_dim, pairwise_state_dim):
        super().__init__()

        self.layernorm = nn.LayerNorm(sequence_state_dim)
        self.proj = nn.Linear(sequence_state_dim, inner_dim * 2, bias=True)
        self.o_proj = nn.Linear(2 * inner_dim, pairwise_state_dim, bias=True)

        torch.nn.init.zeros_(self.proj.bias)
        torch.nn.init.zeros_(self.o_proj.bias)

    def forward(self, sequence_state):
        """
        Inputs:
          sequence_state: B x L x sequence_state_dim

        Output:
          pairwise_state: B x L x L x pairwise_state_dim

        Intermediate state:
          B x L x L x 2*inner_dim
        """

        assert len(sequence_state.shape) == 3

        s = self.layernorm(sequence_state)
        s = self.proj(s)
        q, k = s.chunk(2, dim=-1)

        prod = q[:, None, :, :] * k[:, :, None, :]
        diff = q[:, None, :, :] - k[:, :, None, :]

        x = torch.cat([prod, diff], dim=-1)
        x = self.o_proj(x)

        return x
    


In [ ]:
# This is code for pair to sequence
# use pair info as bais for transformer attention matrix
class PairToSequence(nn.Module):
    def __init__(self, pairwise_state_dim, num_heads):
        super().__init__()

        self.layernorm = nn.LayerNorm(pairwise_state_dim)
        self.linear = nn.Linear(pairwise_state_dim, num_heads, bias=False)

    def forward(self, pairwise_state):
        """
        Inputs:
          pairwise_state: B x L x L x pairwise_state_dim

        Output:
          pairwise_bias: B x L x L x num_heads
        """
        assert len(pairwise_state.shape) == 4
        z = self.layernorm(pairwise_state)
        pairwise_bias = self.linear(z)
        return pairwise_bias


In [ ]:

# costomized Dropout so that it consistantly applied to same dimension 
# for exmaple, along 0, so that each batch has the same dropout to improve learning
import typing as T
class Dropout(nn.Module):
    """
    Implementation of dropout with the ability to share the dropout mask
    along a particular dimension.
    """

    def __init__(self, r: float, batch_dim: T.Union[int, T.List[int]]):
        super(Dropout, self).__init__()

        self.r = r
        if type(batch_dim) == int:
            batch_dim = [batch_dim]
        self.batch_dim = batch_dim
        self.dropout = nn.Dropout(self.r)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        shape = list(x.shape)
        if self.batch_dim is not None:
            for bd in self.batch_dim:
                shape[bd] = 1
        return x * self.dropout(x.new_ones(shape))

In [5]:
# example of dropout function

import torch
import torch.nn as nn

# Define input tensor (batch size = 4, sequence length = 3, feature dim = 2)
x = torch.tensor(
    [[[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]],
     [[7.0, 8.0], [9.0, 10.0], [11.0, 12.0]],
     [[13.0, 14.0], [15.0, 16.0], [17.0, 18.0]],
     [[19.0, 20.0], [21.0, 22.0], [23.0, 24.0]]]
)

# Create the dropout layer with r=0.5 and shared mask along batch dimension
dropout_layer = Dropout(r=0.5, batch_dim=0)

# Set random seed for reproducibility
torch.manual_seed(42)

# Apply dropout
output = dropout_layer(x)

print("Input Tensor:")
print(x)
print("\nOutput Tensor:")
print(output)

Input Tensor:
tensor([[[ 1.,  2.],
         [ 3.,  4.],
         [ 5.,  6.]],

        [[ 7.,  8.],
         [ 9., 10.],
         [11., 12.]],

        [[13., 14.],
         [15., 16.],
         [17., 18.]],

        [[19., 20.],
         [21., 22.],
         [23., 24.]]])

Output Tensor:
tensor([[[ 2.,  4.],
         [ 6.,  8.],
         [ 0., 12.]],

        [[14., 16.],
         [18., 20.],
         [ 0., 24.]],

        [[26., 28.],
         [30., 32.],
         [ 0., 36.]],

        [[38., 40.],
         [42., 44.],
         [ 0., 48.]]])


In [6]:
# Special Attetnion units
#1. add bias from pair info 2.with gating option 3. with masking option
class Attention(nn.Module):
    def __init__(self, embed_dim, num_heads, head_width, gated=False):
        super().__init__()
        assert embed_dim == num_heads * head_width

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_width = head_width

        self.proj = nn.Linear(embed_dim, embed_dim * 3, bias=False)
        self.o_proj = nn.Linear(embed_dim, embed_dim, bias=True)
        self.gated = gated
        if gated:
            self.g_proj = nn.Linear(embed_dim, embed_dim)
            torch.nn.init.zeros_(self.g_proj.weight)
            torch.nn.init.ones_(self.g_proj.bias)

        self.rescale_factor = self.head_width**-0.5

        torch.nn.init.zeros_(self.o_proj.bias)

    def forward(self, x, mask=None, bias=None, indices=None):
        """
        Basic self attention with optional mask and external pairwise bias.
        To handle sequences of different lengths, use mask.

        Inputs:
          x: batch of input sequneces (.. x L x C)
          mask: batch of boolean masks where 1=valid, 0=padding position (.. x L_k). optional.
          bias: batch of scalar pairwise attention biases (.. x Lq x Lk x num_heads). optional.

        Outputs:
          sequence projection (B x L x embed_dim), attention maps (B x L x L x num_heads)
        """

        t = rearrange(self.proj(x), "... l (h c) -> ... h l c", h=self.num_heads)
        q, k, v = t.chunk(3, dim=-1)

        q = self.rescale_factor * q
        a = torch.einsum("...qc,...kc->...qk", q, k)

        # Add external attention bias.
        if bias is not None:
            a = a + rearrange(bias, "... lq lk h -> ... h lq lk")

        # Do not attend to padding tokens.
        if mask is not None:
            mask = repeat(
                mask, "... lk -> ... h lq lk", h=self.num_heads, lq=q.shape[-2]
            )
            a = a.masked_fill(mask == False, -np.inf)

        a = F.softmax(a, dim=-1)

        y = torch.einsum("...hqk,...hkc->...qhc", a, v)
        y = rearrange(y, "... h c -> ... (h c)", h=self.num_heads)

        if self.gated:
            y = self.g_proj(x).sigmoid() * y
        y = self.o_proj(y)

        return y, rearrange(a, "... lq lk h -> ... h lq lk")

